In [8]:
using NearestNeighbors
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Base.Collections
using WordStreams
using DataStructures
using Clustering
using Distances
using Iterators
using StatsBase
using Training

In [5]:
guts = open(deserialize,"guts.jsz","r");

In [9]:
map(println, keys(guts));

init_learning_rate
codebook
network_type
initial_nsenses
dimension
classification_tree
distribution
lsize
init_type
rsize
min_count
subsampling
force_minibatch_size
iter
min_count_for_multiple_senses
corpus_size
embedding


In [11]:
ee=FixedWordSenseEmbedding(
guts[:embedding],
guts[:classification_tree],
guts[:distribution],
guts[:codebook],
DefaultDict(AbstractString,DefaultDict{Int64,Int64},()->DefaultDict(Int64,Int64,()->0)),
guts[:force_minibatch_size],
guts[:init_type],
guts[:network_type],
guts[:dimension],
guts[:lsize],
guts[:rsize],
guts[:corpus_size],
guts[:subsampling],
guts[:init_learning_rate],
guts[:iter],
guts[:min_count],
guts[:min_count_for_multiple_senses],
guts[:initial_nsenses]
)

In [18]:
ee1=ee;

In [16]:
save(ee,"models/ss/text8_200__i5_v2.model")

In [ ]:
ee = restore("models/ss/text8_200__i5_v2.model");
dtree,labels = nn_tree(ee)

In [7]:
guts = Dict{Symbol,Any}()
for name in fieldnames(ee)
    guts[name] = getfield(ee,name)
end

In [9]:
open("guts.jsz","w") do fh
    serialize(fh,guts)
end

In [1]:
DefaultDict(AbstractString,DefaultDict{Int64,Int64},()->DefaultDict(Int64,Int64,()->0))

LoadError: LoadError: UndefVarError: DefaultDict not defined
while loading In[1], in expression starting on line 1

In [5]:
@time find_nearest_words(ee,"solvent",dtree,labels )

LoadError: LoadError: UndefVarError: ee not defined
while loading In[5], in expression starting on line 155

In [6]:
?readdlm

search: readdlm readdir readandwrite



```
readdlm(source, delim::Char, T::Type, eol::Char; header=false, skipstart=0, skipblanks=true, use_mmap, quotes=true, dims, comments=true, comment_char='#')
```

Read a matrix from the source where each line (separated by `eol`) gives one row, with elements separated by the given delimiter. The source can be a text file, stream or byte array. Memory mapped files can be used by passing the byte array representation of the mapped segment as source.

If `T` is a numeric type, the result is an array of that type, with any non-numeric elements as `NaN` for floating-point types, or zero. Other useful values of `T` include `String`, `AbstractString`, and `Any`.

If `header` is `true`, the first row of data will be read as header and the tuple `(data_cells, header_cells)` is returned instead of only `data_cells`.

Specifying `skipstart` will ignore the corresponding number of initial lines from the input.

If `skipblanks` is `true`, blank lines in the input will be ignored.

If `use_mmap` is `true`, the file specified by `source` is memory mapped for potential speedups. Default is `true` except on Windows. On Windows, you may want to specify `true` if the file is large, and is only read once and not written to.

If `ignore_invalid_chars` is `true`, bytes in `source` with invalid character encoding will be ignored. Otherwise an error is thrown indicating the offending character position.

If `quotes` is `true`, column enclosed within double-quote (") characters are allowed to contain new lines and column delimiters. Double-quote characters within a quoted field must be escaped with another double-quote.  Specifying `dims` as a tuple of the expected rows and columns (including header, if any) may speed up reading of large files.  If `comments` is `true`, lines beginning with `comment_char` and text following `comment_char` in any line are ignored.

```
readdlm(source, delim::Char, eol::Char; options...)
```

If all data is numeric, the result will be a numeric array. If some elements cannot be parsed as numbers, a cell array of numbers and strings is returned.

```
readdlm(source, delim::Char, T::Type; options...)
```

The end of line delimiter is taken as `n`.

```
readdlm(source, delim::Char; options...)
```

The end of line delimiter is taken as `n`. If all data is numeric, the result will be a numeric array. If some elements cannot be parsed as numbers, a cell array of numbers and strings is returned.

```
readdlm(source, T::Type; options...)
```

The columns are assumed to be separated by one or more whitespaces. The end of line delimiter is taken as `n`.

```
readdlm(source; options...)
```

The columns are assumed to be separated by one or more whitespaces. The end of line delimiter is taken as `n`. If all data is numeric, the result will be a numeric array. If some elements cannot be parsed as numbers, a cell array of numbers and strings is returned.


In [7]:
function load_wordsim353(filename="data/corpora/wordsim353/combined.csv")
    wordsims = readdlm(filename, ','; skipstart=1)
    wordpairs = convert(Matrix{String},wordsims[:,1:2])
    groundsim = convert(Vector{Float64},wordsims[:,3])
    (wordpairs, groundsim)
end

load_wordsim353 (generic function with 2 methods)

In [8]:
function get_sims(ee::FixedWordSenseEmbedding, wordpairs::Matrix{String})
    sims = Vector{Float64}(size(wordpairs,1))
    for ii in 1:size(wordpairs,1)
        try
            wvs_1 = hcat(ee.embedding[wordpairs[ii,1]]...)
            wvs_2 = hcat(ee.embedding[wordpairs[ii,2]]...)
            sims[ii]=1-minimum(pairwise(CosineDist(), wvs_1,wvs_2))
        catch ex
            warn(ex)
            sims[ii]=NaN
        end
    end
    return sims
end
sims = get_sims(ee,map(lowercase,wordpairs))


LoadError: LoadError: UndefVarError: ee not defined
while loading In[8], in expression starting on line 15

In [9]:
corspearman(groundsim,sims)

LoadError: LoadError: UndefVarError: sims not defined
while loading In[9], in expression starting on line 1

In [10]:
get_sims(ee,wordpairs,contexts)

LoadError: LoadError: UndefVarError: ee not defined
while loading In[10], in expression starting on line 1

In [11]:
split("a s s",' ')

3-element Array{SubString{String},1}:
 "a"
 "s"
 "s"

In [12]:

filename = "data/corpora/SCWS/ratings.txt"
function get_context2(fulltext)
    Task() do
        skip_next = 0
        for word in split(fulltext, ' ')
            skip_next-=1
            if word == "<b>"
                skip_next=3 
            end
            skip_next>0 && continue
            produce(word)
        end
    end |> collect
end
entries = readdlm(filename, '\t', String; quotes=false, comments=false)
wordpairs = entries[:,[2,4]]
groundsim = map(s->parse(Float64,s), entries[:,8])
contexts = map(get_context2,entries[:,[6,7]])

2003×2 Array{Array{SubString{String},1},2}:
 SubString{String}["gap","in","income","between","blacks","and","other","non-whites","is","relatively"  …  "Bahia","for","instance","is","80","%","Afro-Brazilian","but","has","never"]                            …  SubString{String}["of","the","neck",",","bridge",",","and","pickups",",","there"  …  "gears","which","the","player","turns","to","change","the","string","tension"]                                  
 SubString{String}["being","elected",",","has","not","used","the","word","genocide","in"  …  ",","Ethiopia",",","Finland",",","France",",","Georgia",",","Germany"]                                                   SubString{String}["Pythagorean","triple",".","A","Pythagorean","consists","of","three","positive","integers"  …  ",","then","so","is","(","\"","ka","\"",",","\""]                                   
 SubString{String}["other","official","languages",",","except","for","English",".","More","than"  …  "contribution","to","building",

In [13]:
function get_sims(ee::FixedWordSenseEmbedding, wordpairs::Matrix{String}, contexts::Matrix)
    sims = Vector{Float64}(size(wordpairs,1))
    for ii in 1:size(wordpairs,1)
        try
            
            sense1 = Training.WSD(wordpairs[ii,1],contexts[ii,1])
            wv1 = ee.embedding[wordpairs[ii,1]][sense1]
            
            sense2 = Training.WSD(wordpairs[ii,2],contexts[ii,2])
            wv2 = ee.embedding[wordpairs[ii,2]][sense2]
            
            sims[ii]=1-cosine_dist(wv1,wv2)
        catch ex
            if typeof(ex)==KeyError
                warn(ex)
                sims[ii]=NaN
            else
                rethrow(ex)
            end
        end
    end
    return sims
end

get_sims (generic function with 2 methods)

In [14]:
get_sims(ee::FixedWordSenseEmbedding, wordpairs::Matrix{String}, contexts::Matrix)

LoadError: LoadError: UndefVarError: ee not defined
while loading In[14], in expression starting on line 1